# Applied Data Science Capstone 
<hr/>

This notebook will be used for the capstone project

In [1]:
import pandas as pd
import numpy as np
import requests
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge geocoder
#import geocoder

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



# Week 3 Peer Graded Assigment
<hr>

### 1. Retrieve Toronto neighborhood  data  
Data are retrieved by scraping an html page with BeautifulSoup library and, secondly, a Pandas dataframe is created, removing each 'Not Assigned' rows. 

In [2]:
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
toronto_neighborhood=pd.DataFrame(table_contents)

toronto_neighborhood.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


#### Observation: 
No Data Preprocessing and Cleaning needed. Indeed, neigborhoods are already grouped by PostalCode (separated by ',')

In [3]:
print("Shape of data: {}".format(toronto_neighborhood.shape))

Shape of data: (103, 3)


### 2. Create a map of Toronto showing the overlapping neighborhoods 
##### 2.1 Download and import data containing latitude and longitude for each postal code. 

In [4]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
print('Download completed.')
geo_coordinates = pd.read_csv('Geospatial_Coordinates.csv') 
geo_coordinates.head()

Download completed.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### 2.2 Merge toronto_neighborhood and geo_coordinates by '_postal code_'

In [5]:
geo_coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)
toronto_neighborhood = pd.merge(toronto_neighborhood,geo_coordinates,on='PostalCode')
toronto_neighborhood.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494



##### 2.3 Create a Folium map in order to visualize data

In [6]:
# Create map of Toronto using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhood['Latitude'], toronto_neighborhood['Longitude'], toronto_neighborhood['Borough'], toronto_neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='#FFFFFF',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### You might not able to see map within github. Please access the following notebook link to view the Folium output correctly: [Jupiter Notebook](https://eu-gb.dataplatform.cloud.ibm.com/analytics/notebooks/v2/3ff59ce2-5080-479c-9f12-74ce4e6c9fe3/view?access_token=da238977f305645d2d4b22efe2c4ce133334cd2f1cb0a1e4401c4488cd60624c)

### 3. Explore and cluster neighborhoods in Toronto
##### 3.1 Filter data having 'Toronto' as Borough

In [7]:
toronto_data = toronto_neighborhood[toronto_neighborhood['Borough'].str.contains('Toronto')]
toronto_data.reset_index(inplace=True, drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


##### 3.2 Retrieve nearby venues based on latitude and longitude

In [8]:
#Let's create a function for retrieving neaby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
CLIENT_ID = 'GPHXEDRD00JSTIGJISXJRJQGV1AOJVEXMV0P1SWIDITGNJCK'
CLIENT_SECRET = 'ORTS3EOYPU3WZUCX50TYZEQPWUWSQLWZDENIH13G5GSRPGT1'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                )
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


##### 3.3 Extrapolate venue category by creating dummie columns

In [10]:
# Let's categorize each 'Venue Category'
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[cols]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 3.4 Group rows by Neighborhood and by taking the mean of the frequency of occurrence of each category

In [11]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0625,0.0625,0.0625,0.0625,0.1875,0.125,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Central Bay Street,0.014493,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.014493,0.0,0.0,0.0,0.0
4,Christie,0.000000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [12]:
# Function for sorting venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### 3.4 Create the new dataframe and display the top 10 venues for each neighborhood.

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Restaurant,Farmers Market,Beer Bar,Pharmacy,Cheese Shop,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bakery,Breakfast Spot,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Performing Arts Venue
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Gate
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Restaurant,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Spa
4,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Restaurant,Athletics & Sports,Italian Restaurant,Nightclub,Baby Store


##### 3.4 Cluser neighboorhoods

In [14]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 1, 4, 2, 2, 2, 2, 2, 2, 0, 3, 2, 2, 2], dtype=int32)

In [15]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_merged with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Spa,Beer Store,Shoe Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Hotel,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Café,Fast Food Restaurant,Department Store,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Clothing Store,Bakery,Cocktail Bar,Cosmetics Shop,Creperie,Lingerie Store,Moroccan Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Restaurant,Farmers Market,Beer Bar,Pharmacy,Cheese Shop,Diner


##### 3.6 Visualize clusterized data within the map

In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### You might not able to see map within github. Please access the following notebook link to view the Folium output correctly: [Jupiter Notebook](https://eu-gb.dataplatform.cloud.ibm.com/analytics/notebooks/v2/3ff59ce2-5080-479c-9f12-74ce4e6c9fe3/view?access_token=da238977f305645d2d4b22efe2c4ce133334cd2f1cb0a1e4401c4488cd60624c)